# Lesson 4 – Conditional Pattern: Customer Support Router Agent

This notebook is part of the **LangGraph Agentic AI – Intro Course**.

In this lesson, you will build a **conditional routing agent** that classifies incoming customer support messages by topic and routes them to specialized handler nodes.


## 1. Objectives & Prerequisites

**Objectives**

By the end of this lesson, you can:

- Design a `TypedDict` state for a support message router.
- Implement a classifier node that sets a `topic` field based on message content.
- Implement multiple handler nodes that generate different responses.
- Use **conditional edges** in a LangGraph to route to different nodes based on `topic`.

**Prerequisites**

- Lesson 1: single-node pattern.
- Lesson 2: multiple inputs state.
- Lesson 3: sequential pattern.
- Comfortable with Python string handling and `if`/`elif`/`else`.


## 2. Environment Setup

Make sure you have installed the course dependencies (from the repo root):

```bash
python -m venv .venv
source .venv/bin/activate   # on Windows: .venv\Scripts\activate
pip install -r env/requirements.txt
```

Then start Jupyter and select the `langgraph-intro` (or equivalent) Python kernel.


## 3. Concept Warm-up

In the previous lesson, you built a **sequential pipeline** for bug triage.

Now we introduce **conditional branching**:

- A **classifier node** inspects the state and chooses a `topic`.
- Based on `topic`, the graph routes to **different handler nodes**.
- Each handler node can produce a specialized response.

Example messages:

- Billing:  
  > "I need a refund for my last invoice."

- Technical:  
  > "My app shows an error when I click save."

- Account:  
  > "I forgot my password."

Our agent should:

1. Classify the `user_message` into a `topic` like `"billing"`, `"technical"`, `"account"`, or `"unknown"`.
2. Route to the appropriate handler.
3. Write a `response` string into the state.


### Scratch cell (optional)

Use this cell for quick string experiments while you follow along.


In [ ]:
# Scratch space for experimentation

message = "I need a refund for my last invoice."
message.lower()

## 4. Define the State

We define a simple `SupportState` to carry the message, its topic, and the generated response:

- `user_message: str` – the incoming text from the user.
- `topic: str` – one of `"billing"`, `"technical"`, `"account"`, or `"unknown"`.
- `response: str` – the handler's reply.


In [ ]:
from typing import TypedDict

class SupportState(TypedDict):
    user_message: str
    topic: str
    response: str

# Example initial state
initial_state: SupportState = {
    "user_message": "I need a refund for my last invoice.",
    "topic": "",
    "response": "",
}

initial_state

## 5. Classifier Node – `classify_topic_node`

This node will:

1. Lowercase the `user_message`.
2. Use simple keyword rules to set `topic`:
   - Billing: contains `"invoice"`, `"refund"`, or `"payment"`.
   - Technical: contains `"error"`, `"bug"`, or `"crash"`.
   - Account: contains `"password"`, `"username"`, or `"login"`.
   - Otherwise: `"unknown"`.


In [ ]:
def classify_topic_node(state: SupportState) -> SupportState:
    """Classify the user message into a topic based on simple keywords."""
    text = state["user_message"].lower()

    if any(w in text for w in ["invoice", "refund", "payment"]):
        topic = "billing"
    elif any(w in text for w in ["error", "bug", "crash"]):
        topic = "technical"
    elif any(w in text for w in ["password", "username", "login"]):
        topic = "account"
    else:
        topic = "unknown"

    state["topic"] = topic
    return state


# Quick test
state_test = initial_state.copy()
classify_topic_node(state_test)

## 6. Handler Nodes

We implement one handler node per topic:

- `billing_handler_node`
- `technical_handler_node`
- `account_handler_node`
- `fallback_handler_node` (for `"unknown"`)


In [ ]:
def billing_handler_node(state: SupportState) -> SupportState:
    state["response"] = (
        "Thanks for reaching out about billing. "
        "Please share your invoice number so we can assist."
    )
    return state


def technical_handler_node(state: SupportState) -> SupportState:
    state["response"] = (
        "Thanks for reporting a technical issue. "
        "Please send a screenshot and steps to reproduce."
    )
    return state


def account_handler_node(state: SupportState) -> SupportState:
    state["response"] = (
        "For account issues, please use the password reset link "
        "or confirm your email address so we can help."
    )
    return state


def fallback_handler_node(state: SupportState) -> SupportState:
    state["response"] = (
        "We're not sure which team should handle this. "
        "A human agent will review your message shortly."
    )
    return state

## 7. Build the Conditional LangGraph

We now build a `StateGraph` with conditional routing.

Steps:

1. Add all nodes.
2. Set `classify_topic_node` as the entry point.
3. From `classify_topic_node`, add **conditional edges** that route based on `topic`.
4. Each handler node ends the graph.


In [ ]:
from langgraph.graph import StateGraph, END

# 1. Create the graph
graph = StateGraph(SupportState)

# 2. Add nodes
graph.add_node("classify_topic", classify_topic_node)
graph.add_node("billing_handler", billing_handler_node)
graph.add_node("technical_handler", technical_handler_node)
graph.add_node("account_handler", account_handler_node)
graph.add_node("fallback_handler", fallback_handler_node)

# 3. Set entry point
graph.set_entry_point("classify_topic")

# 4. Add conditional edges based on state["topic"]

def route_by_topic(state: SupportState):
    return state["topic"]

# Map topic values to next nodes
graph.add_conditional_edges(
    "classify_topic",
    route_by_topic,
    {
        "billing": "billing_handler",
        "technical": "technical_handler",
        "account": "account_handler",
        "unknown": "fallback_handler",
    },
)

# 5. Handlers go to END
graph.add_edge("billing_handler", END)
graph.add_edge("technical_handler", END)
graph.add_edge("account_handler", END)
graph.add_edge("fallback_handler", END)

# 6. Compile the graph
app = graph.compile()

# 7. Test with a few messages

examples = [
    "I need a refund for my last invoice.",
    "My app shows an error when I click save.",
    "I forgot my password.",
    "Do you ship to Canada?",
]

for msg in examples:
    result = app.invoke({"user_message": msg, "topic": "", "response": ""})
    print("Message:", msg)
    print("Topic:", result["topic"])
    print("Response:", result["response"])
    print("-" * 40)

## 8. Exercises

Use the cells below to implement the exercises for this lesson.

---

### Exercise 1 – Add a “shipping” Topic

**Goal:**

Add a new topic: `"shipping"`.

- Keywords: `"shipping"`, `"delivery"`, `"tracking"`.
- Create a `shipping_handler_node`.
- Update `classify_topic_node` and the conditional routing so that messages with those keywords go to the new handler.
- The handler should set an appropriate `response` string (e.g. asking for order ID).


In [ ]:
# TODO: Add a shipping topic and handler.

# Hint:
# 1. Update classify_topic_node to set topic = "shipping" for shipping-related keywords.
# 2. Implement shipping_handler_node(state: SupportState) -> SupportState.
# 3. Add the node to the graph and extend the conditional edges mapping.


### Exercise 2 – Improve Classifier Priority

**Goal:**

Sometimes a message may match multiple keyword sets, for example:

> "My payment failed and now I see an error on checkout."

This might match both billing and technical.

Implement a consistent priority rule, e.g.:

- If a message contains **billing** and **technical** keywords:
  - Billing wins (or whichever you choose).

Update `classify_topic_node` so that the logic reflects clear priorities rather than the implicit order of `if`/`elif` checks.


In [ ]:
# TODO: Improve classifier to apply an explicit priority when multiple topics match.

# Hint:
# - Compute boolean flags like is_billing, is_technical, is_account first.
# - Then apply if/elif logic using those flags in your chosen priority order.


### Stretch Exercise 1 – Multi-topic Detection

**Goal:**

Extend the state to support **multiple topics**.

1. Add a new field:

   ```python
   topics: List[str]
   ```

2. Modify `classify_topic_node` so that it collects all matching topics into `topics`, while still setting a primary `topic` for routing.
3. Optionally, update handler responses to mention secondary topics if `len(topics) > 1`.


In [ ]:
# TODO: Extend SupportState with a 'topics: List[str]' field and collect all matches.

# Hint:
# from typing import List
# state["topics"] = []
# if is_billing: state["topics"].append("billing")
# etc.


### Stretch Exercise 2 – Language Fallback

**Goal:**

Detect very simple non-English messages and route them to the fallback handler with a special note.

Ideas:

- If the message contains obvious non-English words (e.g. "bonjour", "hola"):
  - Set `topic = "unknown"`.
  - In `fallback_handler_node`, if such words are detected, adjust the response to mention limited language support.

This is intentionally naive, just to show how the router could react to language hints.


In [ ]:
# TODO: Add a naive language check to route some messages to fallback with a special note.

# Hint:
# if any(word in text for word in ["bonjour", "hola"]):
#     topic = "unknown"
#     # later, in fallback_handler_node, adjust response based on these words.


---

You’ve now completed the scaffold for **Lesson 4 – Conditional Pattern**.

Next steps:

- Commit this notebook and your solutions to your Git repo.
- Continue to Lesson 5, where you will build a **looping agent** that iteratively refines a social media post.
